<a href="https://colab.research.google.com/github/TC49501/genaiml/blob/main/hf_rag_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install transformers datasets faiss-cpu sentence-transformers


In [31]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [32]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
documents = ["Kubernetes is an open-source container orchestration platform.",
             "Docker helps package applications in lightweight containers."]

embeddings = embedder.encode(documents)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))


In [37]:
def retrieve_answer(query, index, documents):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), k=1)
    return documents[indices[0][0]]

query = "What is Kubernetes?"
retrieved_text = retrieve_answer(query, index, documents)


In [38]:
inputs = tokenizer(retrieved_text, return_tensors="pt", truncation=True)
outputs = model.generate(**inputs)
final_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Answer:", final_answer)


Answer: Kubernetes is an open-source container orchestration platform. It is used by Docker, Kubernetes, Docker Swarm and other Docker-based containers. It was created by Google in 2007. It has been used by Google for its cloud-based cloud services since 2008.
